In [82]:
import numpy as np
import pandas as pd
import plotly.express as px

import DataRetriever as dr

retriever = dr.DataRetriever()

year_two = retriever.get_data("All-Subsystems-minute-Year2.pkl")

pd.options.mode.chained_assignment = None

In [83]:
# Convert column from string to datetime.
year_two["Timestamp"] = pd.to_datetime(year_two["Timestamp"])

In [84]:
# Creating a pd.Series with the timestamp shifted one downwards. Adding the first value twice, and excluding the last.
timestamp_plus_one = pd.concat([pd.Series(year_two["Timestamp"][0]), year_two["Timestamp"][:-1]], ignore_index=True)

# Creating a new column with the time delta in seconds.
year_two["Timestamp_Delta"] = (year_two["Timestamp"] - timestamp_plus_one).astype('timedelta64[s]')
df_delta = year_two[["Timestamp" ,"Timestamp_Delta"]][1:]
print(df_delta[df_delta['Timestamp_Delta'] < 60])

                        Timestamp  Timestamp_Delta
30      2015-02-01 00:31:49-05:00             59.0
270     2015-02-01 04:31:49-05:00             59.0
870     2015-02-01 14:31:48-05:00             59.0
1468    2015-02-02 00:31:41-05:00             59.0
1768    2015-02-02 05:31:41-05:00             59.0
...                           ...              ...
516998  2016-01-30 04:31:28-05:00             59.0
517598  2016-01-30 14:31:27-05:00             59.0
518196  2016-01-31 00:31:23-05:00             57.0
518406  2016-01-31 04:01:25-05:00             59.0
519006  2016-01-31 14:01:24-05:00             59.0

[1079 rows x 2 columns]


What information can be extracted from binning the intervals?

In [85]:
df_delta.groupby(pd.cut(df_delta["Timestamp_Delta"], bins=[0, 54, 59, 60, 65, 300, np.inf])).count()

,Timestamp,Timestamp_Delta
Timestamp_Delta,,
"(0.0, 54.0]",0,0
"(54.0, 59.0]",1079,1079
"(59.0, 60.0]",517762,517762
"(60.0, 65.0]",390,390
"(65.0, 300.0]",369,369
"(300.0, inf]",3,3


We see that 762 intervals are larger than 1 minute, and 1080 intervals smaller than 1 minute.
Is it possible to recognise a pattern for the missing intervals?

In [86]:
df_delta["Bin"] = pd.cut(df_delta["Timestamp_Delta"], bins=[0, 54, 59, 60, 65, 300, np.inf])
non_60 = df_delta.loc[df_delta["Timestamp_Delta"] != 60]

In [87]:
# # Correct Timestamps
# for row in non_60.index:
#     dt = df_delta["Timestamp"][row]
#     non_60.at[row, "Timestamp"] = pd.Timestamp(year=dt.year, month=dt.month, day=dt.day,
#                                        hour=dt.hour, minute=dt.minute, second=dt.second)
# non_60["Timestamp"] = pd.to_datetime(non_60["Timestamp"])
# non_60["Timestamp"] = non_60["Timestamp"].dt.time

# Incorrect Timestamps; however, plot requires incorrect timestamp.
# (Kan ikke kun vise tiden, og datoen skal være den samme for at kunne lave groupby.)
for row in non_60.index:
    dt = df_delta["Timestamp"][row]
    non_60.at[row, "Timestamp"] = pd.Timestamp(year=2000, month=1, day=1,
                                               hour=dt.hour, minute=dt.minute, second=00)
non_60["Timestamp"] = pd.to_datetime(non_60["Timestamp"])

In [88]:
non_60

,Timestamp,Timestamp_Delta,Bin
30,2000-01-01 00:31:00,59.0,"(54.0, 59.0]"
60,2000-01-01 01:01:00,61.0,"(60.0, 65.0]"
270,2000-01-01 04:31:00,59.0,"(54.0, 59.0]"
870,2000-01-01 14:31:00,59.0,"(54.0, 59.0]"
1438,2000-01-01 00:01:00,174.0,"(65.0, 300.0]"
...,...,...,...
518166,2000-01-01 00:01:00,179.0,"(65.0, 300.0]"
518196,2000-01-01 00:31:00,57.0,"(54.0, 59.0]"
518226,2000-01-01 01:01:00,63.0,"(60.0, 65.0]"
518406,2000-01-01 04:01:00,59.0,"(54.0, 59.0]"


In [89]:
binned_count = pd.DataFrame(non_60.groupby(["Bin", "Timestamp"]).count()).reset_index()
binned_count = binned_count[binned_count["Timestamp_Delta"] != 0]

In [101]:
fig = px.scatter(binned_count, x="Timestamp", y="Timestamp_Delta", color="Bin", title="Occurrence of Missing Record Given Time of Day", )

fig.update_layout(
    xaxis_title="Time of Day",
    yaxis_title="Count of Premature Records",
)

fig.show()

It was seen that three intervals were larger than 5 minutes, how long (in hours), did the interval last?

In [91]:
df_delta["Timestamp_Delta"].loc[(df_delta["Timestamp_Delta"] > 300)] / 60 / 60
# Hours of missing data for the three longest periods.

375267    72.061389
388327     1.016667
511699    15.107222
Name: Timestamp_Delta, dtype: float64

# Pattern in deltas?

In [92]:
for row in df_delta.index:
    dt = df_delta["Timestamp"][row]
    df_delta.at[row, "Timestamp"] = pd.Timestamp(year=dt.year, month=dt.month, day=dt.day,
                                           hour=dt.hour, minute=dt.minute, second=dt.second)

df_delta["Timestamp"] = pd.to_datetime(df_delta["Timestamp"])

In [93]:
minus_5 = df_delta[df_delta["Bin"] == pd.Interval(54.0, 59.0, closed='right')]
plus_5 = df_delta[df_delta["Bin"] == pd.Interval(60.0, 65.0, closed='right')]

In [94]:
minus_5.drop(["Bin", "Timestamp_Delta"], axis=1, inplace=True)
plus_5.drop(["Bin", "Timestamp_Delta"], axis=1, inplace=True)

In [95]:
minus_5.sort_values("Timestamp", inplace=True)
plus_5.sort_values("Timestamp", inplace=True)

In [96]:
minus_5.reset_index(inplace=True, drop=True)
plus_5.reset_index(inplace=True, drop=True)

In [97]:
minus_5["Count"] = 1
minus_5

,Timestamp,Count
0,2015-02-01 00:31:49,1
1,2015-02-01 04:31:49,1
2,2015-02-01 14:31:48,1
3,2015-02-02 00:31:41,1
4,2015-02-02 05:31:41,1
...,...,...
1074,2016-01-30 04:31:28,1
1075,2016-01-30 14:31:27,1
1076,2016-01-31 00:31:23,1
1077,2016-01-31 04:01:25,1


In [98]:
fig = px.scatter(minus_5, x="Timestamp", y="Count")

fig.show()

In [99]:
(minus_5 - minus_5.shift()).astype('timedelta64[s]')

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer